In [13]:
import Rust_Machine_Learning
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [14]:
# Get the csv data
# df = pd.read_csv("../data/iris.data")
df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.DataFrame({
#   "Dosage": [0.0,2.0,4.0,8.0,11.0,13.0,15.0,18.0,20.0,22.0,25.0,26.0,27.0,28.0,29.0,31.0,34.0,36.0,37.0],
#   "Effectiveness": [0.0,0.0,0.0,0.0,5.0,18.0,100.0,100.0,100.0,100.0,60.0,58.0,56.0,52.0,48.0,15.0,0.0,0.0,0.0]
# })
df  


,x,y,cluster
0,1.000070,40.9378,0
1,0.997360,41.1714,0
2,0.134799,41.8113,0
3,2.475850,41.6346,0
4,-3.058700,41.3887,0
...,...,...,...
2985,98.255900,68.0158,9
2986,96.500700,67.9212,9
2987,96.885200,68.1787,9
2988,96.834000,67.9841,9


In [15]:
# Ensure categoriical data is one-hot encoded
# columns is list of ints
def one_hot_encode(df, columns):
    encoded = df.copy()
    feature_to_cols = dict()
    for col in columns:
        # Get the unique values
        unique_vals = encoded.iloc[:, col].unique()

        # Start col
        start_col = encoded.shape[1]-1

        # Now for each unique value add a col to df
        for val in unique_vals:
            col_encoded = encoded.iloc[:, col] == val
            col_encoded = [(1.0 if label == True else 0.0) for label in col_encoded]
            encoded["is_" + encoded.columns[col] + "_" + str(val)] = col_encoded
        
        end_col = encoded.shape[1]-1
        feature_to_cols[col] = list(range(start_col, end_col))

    encoded.drop(encoded.columns[columns], axis=1, inplace=True)
    return (encoded, feature_to_cols)
to_encode = [2]
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded


,x,y,is_cluster_0,is_cluster_1,is_cluster_2,is_cluster_3,is_cluster_4,is_cluster_5,is_cluster_8,is_cluster_7,is_cluster_9
0,1.000070,40.9378,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.997360,41.1714,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.134799,41.8113,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.475850,41.6346,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-3.058700,41.3887,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2985,98.255900,68.0158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2986,96.500700,67.9212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2987,96.885200,68.1787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2988,96.834000,67.9841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
# Split the data into train and test
def split_train_test(df, percent_train, randomize = True):
  #Randomize the rows
  if randomize:
    df = df.sample(frac=1.0)

  #Create train and test data
  df_size = df.shape[0]
  train_size = df_size * percent_train

  train_data = df.iloc[:int(train_size),:]
  test_data = df.iloc[int(train_size):,:]
  return (train_data, test_data)

In [17]:
def remove_encoded_category_data_and_label(encoded, feature_to_encoded_cols, label_col):
  cols_to_remove = []
  did_remove_label_col = False
  for k,v in feature_to_encoded_cols.items():
    if label_col in v:
      encoded.drop(encoded.columns[v], axis=1, inplace=True)
      did_remove_label_col = True
  if not did_remove_label_col:
    encoded.drop(encoded.columns[label_col], axis=1, inplace=True)
    

In [18]:
# Now setup the train/test data
# We will train it to predict y based on x and cluster
label_col = 1

train_data, test_data = split_train_test(encoded, 0.7)

# Feature Data
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

# Label Data
train_data_labels = train_data[train_data.columns[label_col]].values.tolist()
train_data_features

,x,is_cluster_0,is_cluster_1,is_cluster_2,is_cluster_3,is_cluster_4,is_cluster_5,is_cluster_8,is_cluster_7,is_cluster_9
2505,17.06050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
529,47.59170,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,94.58280,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
661,85.54410,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2021,46.58730,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2876,53.50010,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
405,47.89960,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,48.42270,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
738,86.55500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Now train the model
datapoints_per_node = 25
trained_model = Rust_Machine_Learning.RegressionTree(train_data_features_list, train_data_labels, datapoints_per_node)
trained_model.print()

 7 0.5
   8 0.5
     0 65.84075
       1 0.5
         0 8.539805000000001
           0 8.034089999999999
             0 5.55422
               0 4.033765
                 0 -10.048045
                   -5.77052
                   0 -9.656715
                     3.61729
                     0 -9.56908
                       -4.2444
                       0 3.86393
                         0 -5.68542
                           -0.040103283333333316
                           0 -3.525035
                             2.117917388888889
                             0 -3.44917
                               -6.40089
                               0 -3.2977350000000003
                                 4.682345
                                 0 -3.23184
                                   -5.72386
                                   0 -2.8829399999999996
                                     2.196136
                                     0 -1.68351
                                       -0.75198

In [20]:
# Test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features_list = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.classify(test_data_features_list)

In [22]:
# Graph labelled data by algorithm
result = test_data_features.copy()
result["y"] = labelled_results

fig = px.scatter(result, x="x", y="y")
fig.layout.title = "Labelled Result"
fig.show()
fig2 = px.scatter(test_data, x="x", y="y")
fig2.layout.title = "Expected Result"
fig2.show()